In [ ]:
cd IMGS-789-Advanced-Robust-ML-for-Interdisciplinary-Imaging-Science-Applications/group_project/

/home/ckb2084/research/IMGS-789-Advanced-Robust-ML-for-Interdisciplinary-Imaging-Science-Applications/group_project


/home/ckb2084/conda/envs/pytorch/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Neccesary imports
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from time import time
import random
import glob, os

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from scipy.ndimage.filters import gaussian_filter1d
import torchvision.models as models
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from accelerate import Accelerator
cm = plt.get_cmap('RdYlBu')
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import LayerNorm
import math
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.colors as mcolors

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde


/tmp/ipykernel_1705367/2093688170.py:17: DeprecationWarning: Please import `gaussian_filter1d` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter1d


In [3]:
%%time
# Import all the data

data = pd.read_pickle('grism_specPT.pkl')
data_sed = pd.read_pickle('sed_fitting.pkl')
# data_photo = pd.read_pickle('photo_data.pkl')

CPU times: user 1.33 s, sys: 2.47 s, total: 3.8 s
Wall time: 14.1 s


In [4]:
data_sed['best.chi_square'][3]

np.float64(232.9991061854824)

In [15]:
np.sqrt(data_sed['best.chi_square'])

0             NaN
1       12.121545
2       12.427232
3       15.264308
4       12.558336
          ...    
5968     2.206474
5969     1.183323
5970     0.965716
5971     1.914446
5972     0.305017
Name: best.chi_square, Length: 5973, dtype: float64

In [ ]:
data_sed_chi_l5 = data_sed[data_sed['best.redchi_square'] <= 5.0]
data_sed_chi_l5

,id,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.sfh.sfr100Myrs,bayes.sfh.sfr100Myrs_err,bayes.sfh.sfr10Myrs,bayes.sfh.sfr10Myrs_err,bayes.CTIO_MosaicII.U,bayes.CTIO_MosaicII.U_err,bayes.KPNOU,...,best.subaru.suprime.NB816,best.subaru.suprime.IB827,best.cfht.wircam.J,best.cfht.wircam.H,best.cfht.wircam.Ks,best.KPNOU,best.CTIO_MosaicII.U,best.ukirt.J,best.ukirt.H,best.ukirt.Ks
1031,0.0,103.423556,57.306127,62.311362,34.981520,105.150274,58.259412,0.002728,0.002030,0.002731,...,0.003328,0.003380,0.009851,0.013055,0.016491,0.002237,0.002206,0.009927,0.012906,0.016440
1032,1.0,14.831638,7.785192,8.749829,4.625168,15.013204,7.901320,0.000660,0.000300,0.000677,...,0.000967,0.000922,0.000894,0.000781,0.000816,0.001226,0.001199,0.000897,0.000789,0.000816
1033,2.0,16.269784,9.253563,9.512016,5.192185,16.512681,9.419896,0.000584,0.000329,0.000584,...,0.000722,0.000725,0.001758,0.001558,0.001096,0.000838,0.000837,0.001728,0.001534,0.001127
1034,3.0,21.366991,11.922256,12.880794,7.282398,21.725772,12.118329,0.000480,0.000366,0.000481,...,0.000661,0.000667,0.001557,0.002254,0.004027,0.000288,0.000287,0.001539,0.002255,0.004051
1035,4.0,41.873706,24.168218,26.424520,15.494340,42.652539,24.549463,0.000604,0.000402,0.000612,...,0.000908,0.000906,0.002741,0.004240,0.006092,0.000642,0.000644,0.002726,0.004247,0.006112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,6047.0,76.736727,23.670309,63.522755,20.811619,77.810823,23.763980,0.000107,0.000014,0.000118,...,0.000560,0.000570,0.001303,0.002737,0.004181,0.000125,0.000113,0.001277,0.002745,0.004187
5969,6054.0,12.970196,5.848309,8.519633,3.633422,13.112897,5.918852,0.000434,0.000051,0.000436,...,0.000583,0.000587,0.001417,0.001508,0.001391,0.000417,0.000410,0.001392,0.001505,0.001393
5970,6042.0,12.168651,5.186683,7.867744,2.853938,12.310875,5.268632,0.000751,0.000086,0.000747,...,0.000947,0.000947,0.002031,0.001735,0.001845,0.000732,0.000736,0.002033,0.001732,0.001861
5971,6051.0,39.028449,16.517668,27.610050,9.801470,39.358946,16.682189,0.000389,0.000042,0.000402,...,0.000900,0.000906,0.002120,0.002453,0.002868,0.000393,0.000380,0.002050,0.002466,0.002872


In [7]:
id = [int(i) for i in data_sed_chi_l5['id']]
print(id)

[0, 1, 2, 3, 4, 5, 7, 8, 9, 11, 14, 19, 20, 18, 21, 22, 23, 26, 24, 27, 28, 29, 31, 32, 30, 35, 34, 33, 36, 37, 39, 38, 41, 43, 44, 46, 48, 50, 51, 52, 56, 54, 58, 57, 63, 66, 62, 65, 67, 68, 80, 71, 69, 73, 70, 86, 72, 76, 75, 77, 78, 79, 74, 82, 81, 84, 87, 88, 89, 90, 92, 91, 94, 95, 96, 97, 98, 99, 110, 101, 102, 104, 103, 105, 106, 107, 108, 109, 112, 111, 113, 115, 114, 116, 118, 117, 119, 120, 123, 122, 126, 128, 129, 132, 130, 131, 134, 133, 135, 136, 137, 140, 138, 139, 141, 142, 143, 144, 145, 147, 146, 150, 151, 152, 153, 156, 155, 154, 157, 159, 162, 161, 160, 163, 164, 167, 166, 168, 170, 171, 173, 172, 169, 175, 177, 176, 178, 174, 179, 183, 185, 184, 186, 187, 189, 193, 192, 194, 195, 196, 197, 200, 198, 199, 201, 202, 203, 205, 204, 207, 210, 212, 213, 211, 215, 217, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 206, 233, 229, 232, 231, 230, 234, 236, 228, 237, 239, 240, 238, 214, 242, 243, 244, 241, 246, 245, 248, 247, 249, 250, 251, 252, 253, 255, 256, 254, 258, 2

In [8]:
data = data.iloc[id]

In [ ]:
len(data[data['SNR']>=5]), len(data_sed_chi_l5)

(4360, 4360)

In [15]:
data = data[data['SNR']>=2.5]
data_subset = data[data['z']<1.7]
data_subset

,grism_id,wavelength,flux,z,SNR,continuum_sub_flux
2,aegis-26-G141_00469,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.43,4.094828,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,aegis-26-G141_00703,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.40,20.695364,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
13,aegis-26-G141_00836,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.56,6.308725,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
16,aegis-26-G141_00910,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.05,4.627907,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
18,aegis-26-G141_00937,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.51,8.418367,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...
18575,uds-23-G141_42763,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002...",1.34,4.185430,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002..."
18576,uds-05-G141_42909,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.45,2.974790,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
18577,uds-25-G141_42927,"[10208.409432389317, 10209.33721533346, 10210....","[0.11972591812810127, 0.11699151816036778, 0.1...",0.99,3.177632,"[0.11903046440555565, 0.11629269513745405, 0.1..."
18578,uds-25-G141_42956,"[10208.409432389317, 10209.33721533346, 10210....","[0.10640179127421547, 0.10492161739585708, 0.1...",0.92,3.127168,"[0.10595676034947163, 0.10447454305889436, 0.1..."


In [5]:
# Split the data into 70% train and 30% temp_test
train_df, temp_test_df = train_test_split(data_subset, test_size=0.3, random_state=42)

# Split the temp_test into 50% test and 50% validation, which is 15% each of the original
test_df, val_df = train_test_split(temp_test_df, test_size=0.5, random_state=42)

print(len(train_df),len(test_df),len(val_df))

6078 1302 1303


In [6]:

from SpecPT import (
    SpecPT, 
    SpecPTForRedshift, 
    CustomLoadDataset_Autoencoder,
    Swish,
    # CustomLoadDataset_Redshift, 
    NMADLoss, 
    evaluate
)

In [8]:
class EnhancedSpecPTForRedshift(nn.Module):
    def __init__(self, pretrained_model, additional_features_dim, output_features=1, num_mlp_blocks=5, mlp_dim=512, dropout_rate=0.2):
        super(EnhancedSpecPTForRedshift, self).__init__()
        
        self.encoder = pretrained_model.transformer_encoder
        self.proj_to_d_model = pretrained_model.proj_to_d_model
        self.forward_conv = pretrained_model.forward_conv
        
        # Fine-tune the last few layers of the encoder
        for param in list(self.encoder.parameters())[-4:]:
            param.requires_grad = True
        
        # Projection layer for additional features
        self.additional_projection = nn.Sequential(
            nn.Linear(additional_features_dim, 512),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.LayerNorm(512)
        )
        
        # Early fusion layer before attention
        self.pre_attention_fusion = nn.Sequential(
            nn.Linear(512 * 2, 512),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.LayerNorm(512)
        )
        
        self.attention = nn.MultiheadAttention(embed_dim=512, num_heads=8)
        
        self.mlp_blocks = nn.Sequential(
            *[ImprovedResidualMLPBlock(mlp_dim, mlp_dim, dropout_rate) for _ in range(num_mlp_blocks)]
        )
        
        self.prediction = nn.Sequential(
            nn.Linear(mlp_dim, mlp_dim // 2),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_dim // 2, output_features),
            nn.Softplus()
        )
        
    def forward(self, x, additional_data):
        # Process spectral data
        x = x.unsqueeze(1)
        x = self.forward_conv(x)
        x = x.flatten(start_dim=1)
        x = self.proj_to_d_model(x)
        x = x.unsqueeze(0)
        
        # Encode spectral features
        encoded_features = self.encoder(x)
        encoded_features = encoded_features.squeeze(0)
        
        # Process and combine additional data
        additional_features = self.additional_projection(additional_data)
        
        # Combine features before attention
        combined_features = torch.cat([encoded_features, additional_features], dim=-1)
        fused_features = self.pre_attention_fusion(combined_features)
        
        # Apply attention mechanism
        attn_output, _ = self.attention(fused_features, fused_features, fused_features)
        x = attn_output + fused_features  # Residual connection
        
        # Process through MLP blocks
        x = self.mlp_blocks(x)
        
        # Final prediction
        redshift = self.prediction(x)
        return redshift

class ImprovedResidualMLPBlock(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate):
        super(ImprovedResidualMLPBlock, self).__init__()
        self.linear1 = nn.Linear(input_dim, output_dim)
        self.linear2 = nn.Linear(output_dim, output_dim)
        self.swish = Swish()
        self.layer_norm = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        residual = x
        x = self.swish(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        x = x + residual  # Residual connection
        x = self.layer_norm(x)
        return self.swish(x)


In [9]:
# Dataset Loader for Redshift
class CustomLoadDataset_Redshift(Dataset):
    def __init__(self, df):
        x = []
        y = []
        target_id = []
            
        for index, row in df.iterrows():
            fl = row['flux']
            if np.median(fl) > 0:
                fl = fl / np.median(fl)
                x.append(fl)
                y.append(np.array([row['z']]))
                target_id.append(row['grism_id'])

        self.X = torch.from_numpy(np.stack(x, axis=0))
        self.Y = torch.from_numpy(np.stack(y, axis=0))
        self.t_id = target_id

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].float(), self.Y[idx].float(), idx, self.t_id[idx]


In [10]:
valid_loader = DataLoader(CustomLoadDataset_Redshift(val_df), batch_size=100, shuffle=True)
test_loader = DataLoader(CustomLoadDataset_Redshift(test_df), batch_size=256, shuffle=False)

In [11]:
n=1
epochs = 800
batch_size = 64
lr = 1e-5 #orig = 0.001 or 1e-3, best with 5e-5

config = {
    "input_size": 7781,
    "d_model": 512,
    "nhead": 8,
    "num_encoder_layers": 3,
    "num_decoder_layers": 3,
    "dim_feedforward": 2048,
    "learning_rate": lr,
    "batch_size": 64,
    "num_mlp_blocks": 5,
    "mlp_dim": 512,
    "dropout_rate": 0.2
}

In [12]:
train_loader = DataLoader(CustomLoadDataset_Redshift(train_df), batch_size=batch_size, shuffle=True)